In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, QuantileTransformer
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

from ember_utils import *
from ember_model import *
from ember_pjr_utils import *

In [11]:
def get_only_av_class_labeled_samples(all_X, all_Y, Y_Family):
    
    new_X = []
    new_Y = []
    new_Y_family = []
    
    for ind, y in enumerate(all_Y):
        if y == 1:
            if Y_Family[ind] != '':
                new_Y.append(y)
                new_X.append(all_X[ind])
                new_Y_family.append(Y_Family[ind])
        else:
            new_Y.append(y)
            new_X.append(all_X[ind])
            new_Y_family.append(Y_Family[ind])
            
            
    print(len(all_Y==1) == (len(new_Y) + (len(all_Y) - len(new_Y))))  
    print(len(new_X) == len(new_Y))
    return new_X, new_Y, new_Y_family

def get_family_labeled_month_data(data_dir, month, train=True):
    
    if train:
        data_dir = data_dir + str(month) + '/'
        XY_train = np.load(data_dir + 'XY_train.npz')
        X_tr, Y_tr, Y_tr_family = XY_train['X_train'], XY_train['Y_train'], XY_train['Y_family_train']

        #print(f'X_train {X_tr.shape} Y_train {Y_tr.shape} Y_tr_family {Y_tr_family.shape}')
        
        return X_tr, Y_tr, Y_tr_family
    else:
        data_dir = data_dir + str(month) + '/'
        XY_test = np.load(data_dir + 'XY_test.npz')
        X_test, Y_test, Y_test_family = XY_test['X_test'], XY_test['Y_test'], XY_test['Y_family_test']

        return X_test, Y_test, Y_test_family

    
    
def get_family_labeled_task_test_data(data_dir, task_months, mlp_net=False):
    
    X_te, Y_te, Y_te_family = get_family_labeled_month_data(data_dir, task_months[-1], train=False)
    
    for month in task_months[:-1]:
        pre_X_te, pre_Y_te, pre_Y_te_family = get_family_labeled_month_data(data_dir, month, train=False)
        X_te, Y_te, Y_te_family = np.concatenate((X_te, pre_X_te)), np.concatenate((Y_te, pre_Y_te)),\
                                np.concatenate((Y_te_family, pre_Y_te_family))
        

    X_test, Y_test, Y_test_family = X_te, Y_te, Y_te_family
    print(f'X_test {X_test.shape} Y_test {Y_test.shape} Y_te_family {Y_te_family.shape}')
    
    return X_test, Y_test, Y_test_family



def get_month_data(data_dir, month, train=True):
    
    if train:
        data_dir = data_dir + str(month) + '/'
        XY_train = np.load(data_dir + 'XY_train.npz')
        X_tr, Y_tr = XY_train['X_train'], XY_train['Y_train']
        
        return X_tr, Y_tr
    else:
        data_dir = data_dir + str(month) + '/'
        XY_test = np.load(data_dir + 'XY_test.npz')
        X_test, Y_test = XY_test['X_test'], XY_test['Y_test']

        return X_test, Y_test 

    
    
def get_MemoryData(X, Y, memory_budget):
    indx = [i for i in range(len(Y))]
    random.shuffle(indx)
    
    replay_index = indx[:memory_budget]
    X_train = X[replay_index]
    Y_train = Y[replay_index]
    
    return X_train, Y_train



def get_GRS_data(data_dir, task_months, memory_budget, train=True, joint=False, none=False):
    
    if train:
        if none:
            X_tr, Y_tr = get_month_data(data_dir, task_months[-1])
            X_train, Y_train  = X_tr, Y_tr
            print(f'X_train {X_train.shape} Y_train {Y_train.shape}\n')
            return X_train, Y_train
        else:
            X_tr, Y_tr = get_month_data(data_dir, task_months[-1])
            print(f'Current Task month {task_months[-1]} data X {X_tr.shape} Y {Y_tr.shape}')

            if len(task_months) != 1:
                previous_Xs, previous_Ys = [], []
                for month in task_months[:-1]:
                    #pre_X_tr, pre_Y_tr, pre_X_val, pre_Y_val = get_month_data(data_dir, month)
                    pre_X_tr, pre_Y_tr = get_month_data(data_dir, month)


                    pre_X_tr, pre_Y_tr = np.array(pre_X_tr), np.array(pre_Y_tr)
                    #print(f'pre_X_tr {pre_X_tr.shape}  pre_Y_tr {pre_Y_tr.shape}')

                    for idx, prevSample in enumerate(pre_X_tr):
                        previous_Xs.append(prevSample)
                        previous_Ys.append(pre_Y_tr[idx])


                previous_Xs, previous_Ys = np.array(previous_Xs), np.array(previous_Ys)  

                #print(f'Y_tr {Y_tr.shape}  previous_Ys {previous_Ys.shape}')
                if joint:
                    X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                else:
                    if memory_budget >= len(previous_Ys):
                        X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                    else:
                        previous_Xs, previous_Ys = get_MemoryData(previous_Xs, previous_Ys, memory_budget)
                        X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                        #print(f'memory_budget {memory_budget}  Y_tr {previous_Ys.shape} ')
                        assert memory_budget == len(previous_Ys)
            

            X_train, Y_train  = X_tr, Y_tr
            print(f'X_train {X_train.shape} Y_train {Y_train.shape}\n')
            return X_train, Y_train
    else:
        #X_te, Y_te = get_month_data(data_dir, task_months[-1], train=False)
        
        X_te, Y_te = [], []
        #for month in task_months[:-1]:
        for month in task_months:
            pre_X_te, pre_Y_te = get_month_data(data_dir, month, train=False)
            
            X_te.append(np.array(pre_X_te)), Y_te.append(np.array(pre_Y_te))
            #X_te, Y_te = np.concatenate((X_te, pre_X_te)), np.concatenate((Y_te, pre_Y_te))

        X_test, Y_test  = X_te, Y_te
        #print(f'X_test {X_test.shape} Y_test {Y_test.shape}')
        return X_test, Y_test


                        
def life_experience(model, X_test, Y_test, batch_size, device):
    result_accs = []
    
    for T, Y in enumerate(Y_test):
        accT, _, _, _ = testing_aucscore(model, X_test[T], Y, batch_size, device)
        result_accs.append(accT)
    
    return np.array(result_accs)

def create_empty_matrix(m, n):
    return np.zeros((m, n))


def testing_aucscore(model, X_test, Y_test, batch_size, device):
    #X_te = torch.from_numpy(X_test).type(torch.FloatTensor)
    #y_te = torch.from_numpy(Y_test).type(torch.FloatTensor) 
    
    testloader = get_dataloader(X_test, Y_test, batch_size, train_data=False)   
    
    model.eval()
    y_pred_list = []
    y_true_list = []
    test_acc = []
    with torch.no_grad():
        for x_batch, y_batch in tqdm(testloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            tmp_test_acc = binary_acc(y_test_pred, y_batch)
            test_acc.append(tmp_test_acc.item())
            
            y_pred_tag = torch.round(y_test_pred).squeeze(1)
            y_pred_list += list(y_pred_tag.cpu().numpy())
            y_true_list += list(y_batch.cpu().numpy())
        
            
    #correct_test_results = (np.array(y_pred_list) == np.array(y_true_list)).sum()
    #acc = correct_test_results/len(y_true_list)
    
#     from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
    
#     correct_labels, predicted_labels = np.array(y_true_list), np.array(y_pred_list)
    
#     roc_auc = roc_auc_score(correct_labels, predicted_labels)
#     precision = precision_score(correct_labels, predicted_labels, average='micro')
#     recall = recall_score(correct_labels, predicted_labels, average='micro')
#     f1score = f1_score(correct_labels, predicted_labels, average='macro')
    
    #print(f'test accuracy {np.mean(test_acc)} and ROC-AUC {roc_auc}')

    #wrong_good, wrong_mal, top_k_mistaken_families = \
    #            get_mistaken_stats(np.array(y_true_list), np.array(np.round(y_pred_list)), Y_test_family, top_k)
    
    #return np.mean(test_acc), roc_auc, precision, recall, f1score
    return np.mean(test_acc)


# parser = argparse.ArgumentParser()
# parser.add_argument('--num_exps', type=int, default=1, required=False, help='Number of Experiments to Run.')
# parser.add_argument('--num_epoch', type=int, default=500, required=False)
# parser.add_argument('--batch_size', type=int, default=6000, required=False)
# parser.add_argument('--grs_joint',  action="store_true", required=False)
# parser.add_argument('--grs_none',  action="store_true", required=False)
# parser.add_argument('--memory_budget', type=int, required=False)
# parser.add_argument('--data_dir', type=str,\
#                     default='../../../month_based_processing_with_family_labels/', required=False)

# args = parser.parse_args()


all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']



# data_dir = args.data_dir
# num_exps = args.num_exps
# num_epoch = args.num_epoch
# batch_size = args.batch_size
# memory_budget = args.memory_budget


data_dir = '../../../month_based_processing_with_family_labels/'
num_exps = 1
num_epoch = 100
batch_size = 6000
memory_budget = 'none'
grs_joint = False

patience = 5



# if args.grs_joint:
#     memory_budget = 'joint'
    
# if args.grs_none:
#     memory_budget = 'none'
    
    
replay_type = 'joint_partial'

exp_seeds = [random.randint(1, 99999) for i in range(num_exps)]


num_tasks = len(all_task_months)
results_matrix = create_empty_matrix(num_tasks, num_tasks)

cnt =  1    
for exp in exp_seeds:
    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net()
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    
    
    
    standardization = StandardScaler()
    standard_scaler = None
    
    count_task = 0
    model_init = 'yes'
    results_acc = []
    results_task = []
    for task_month in range(len(all_task_months)):
                
        print(f'\n{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Round {cnt} ...')
        task_start = time.time()
        current_task = [all_task_months[task_month]]
        task_months = all_task_months[:task_month+1]
        print(f'Current Task {current_task} with Budget {memory_budget}')


        model_save_dir = '../../GRS_SavedModel' + '/GRSModel_' + str(memory_budget) + '/' + str(current_task) + '/'
        create_parent_folder(model_save_dir)
        
        opt_save_path = '../../GRS_SavedModel' + '/GRSOpt_' + str(memory_budget) + '/' + str(current_task) + '/'
        create_parent_folder(opt_save_path)
        
        results_save_dir =  '../../GRS_SavedResults_' +'/GRS_' + str(memory_budget) + '/' 
        create_parent_folder(results_save_dir)
        
        X_train, Y_train = get_GRS_data(data_dir, task_months, memory_budget, train=True, none=True)        
#         if args.grs_joint:
#             X_train, Y_train = get_GRS_data(data_dir, task_months, memory_budget, train=True, joint=True)
#             X_test, Y_test = get_GRS_data(data_dir, all_task_months, memory_budget, train=False, joint=True)
#         elif args.grs_none:
#             X_train, Y_train = get_GRS_data(data_dir, current_task, memory_budget, train=True, joint=True)
#             X_test, Y_test = get_GRS_data(data_dir, all_task_months, memory_budget, train=False, joint=True)
#         else:
#             X_train, Y_train = get_GRS_data(data_dir, task_months, memory_budget, train=True, joint=False)
#             X_test, Y_test = get_GRS_data(data_dir, all_task_months, memory_budget, train=False, joint=False)
    
        
        # to debug
        #X_train = X_train[:500]
        #Y_train = Y_train [:500]
        #X_valid, Y_valid = X_valid[:500], Y_valid[:500]
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
        
        if grs_joint:
                standardization = StandardScaler()
                standard_scaler = None
                standard_scaler = standardization.fit(X_train)
        else:        
                standard_scaler = standardization.partial_fit(X_train)

        X_train = standard_scaler.transform(X_train)
        X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
        
        from sklearn.model_selection import train_test_split
        X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train,\
                                                              test_size=0.10, random_state=42)

        task_training_time, epoch_ran, training_loss, validation_loss  = training_early_stopping(\
                             model, model_save_dir, opt_save_path, X_train, Y_train,\
                             X_valid, Y_valid, patience, batch_size, device, optimizer, num_epoch,\
                             criterion, replay_type, current_task, exp, earlystopping=True)
        
        
        #model = Ember_MLP_Net()
        #model = model.to(device)
        #load the best model for this task
        best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
        print(f'loading best model {best_model_path}')
        model.load_state_dict(torch.load(best_model_path))
        
        
        #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
        best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
        print(f'loading best optimizer {best_optimizer}')
        optimizer.load_state_dict(torch.load(best_optimizer))
        

        for t in range(num_tasks):
            y_t = all_task_months[t]
            X_te, Y_te = get_month_data(data_dir, y_t, train=False)
            X_te, Y_te = np.array(standard_scaler.transform(X_te), np.float32),\
                        np.array(Y_te, np.int32)
            
            acc = testing_aucscore(model, X_te, Y_te, batch_size, device)
            
            results_matrix[task_month][t] = acc
        
        end_time = time.time()

        print(f'Elapsed time {(end_time - start_time)/60} mins.')    


        task_end = time.time()
        task_run_time = (task_end - task_start)/60
        
        
    joint_acc_all = []
    future_acc_all = []
    for t in range(num_tasks):
        joint_acc = np.mean(results_matrix[t][0:t+1])
        future_acc = np.mean(results_matrix[t][t+1:])
        #print(f'task {t+1}\t{results_matrix[t][t+1:]}\n')
        joint_acc_all.append(joint_acc)

        if t+1 != num_tasks:
            #print(future_acc)
            future_acc_all.append(future_acc)

    print(f'Avg-Acc-Till-Current-Task-Joint\t{np.mean(joint_acc_all)}\t\tFuture-Acc\t{np.mean(future_acc_all)}')
    
    end_time = time.time()
    cnt += 1
    print(f'Elapsed time {(end_time - start_time)/60} mins.')

    del model_save_dir
    del opt_save_path
    del results_save_dir


Torch 1.2.0+cu92 CUDA 9.2.148
Model has 3.132161m parameters

2024-05-27 02:01:49 Round 1 ...
Current Task ['2018-01'] with Budget none
X_train (55722, 2381) Y_train (55722,)

2024-05-27 02:01:50 Standardizing ...


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1 of 100


  0%|          | 0/8 [00:00<?, ?it/s]

Train Loss: 0.6996, Train Acc: 0.5332
Val Loss: 0.6825, Val Acc: 0.5536
Validation loss decreased (inf --> 0.682466).  Saving model ...
../../GRS_SavedModel/GRSModel_none/['2018-01']/best_model_epoch_1.pt
../../GRS_SavedModel/GRSOpt_none/['2018-01']/best_optimizer_epoch_1.pt
Epoch 2 of 100


...

100%|██████████| 2/2 [00:00<00:00,  8.63it/s]


Train Loss: 0.0794, Train Acc: 0.9717
Val Loss: 0.1072, Val Acc: 0.9661
EarlyStopping counter: 5 out of 5
Early stopping
Training time: 0.994 minutes
loading best model ../../GRS_SavedModel/GRSModel_none/['2018-12']/best_model_epoch_24.pt
loading best optimizer ../../GRS_SavedModel/GRSOpt_none/['2018-12']/best_optimizer_epoch_24.pt


100%|██████████| 2/2 [00:00<00:00,  5.25it/s]

Elapsed time 11.161363832155864 mins.
Avg-Acc-Till-Current-Task-Joint	0.9317145956078111		Future-Acc	0.913014641641558
Elapsed time 11.161389064788818 mins.



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
# None
Avg-Acc-Till-Current-Task-Joint	0.9317145956078111		Future-Acc	0.913014641641558

# Joint
Avg-Acc-Till-Current-Task-Joint	0.9577413405273368		Future-Acc	0.9245585751963752

array([[0.94164583, 0.90875506, 0.88923669, 0.89959687, 0.90940768,
        0.89051849, 0.91386348, 0.8914814 , 0.87601805, 0.83396575,
        0.87858334, 0.85295835],
       [0.95541668, 0.94661987, 0.91837221, 0.8959493 , 0.91681188,
        0.90477175, 0.93466848, 0.90556902, 0.91537213, 0.85560736,
        0.89612502, 0.88737503],
       [0.95129168, 0.95530105, 0.93859857, 0.90650797, 0.93728226,
        0.91613305, 0.94050163, 0.91789567, 0.92875904, 0.8855893 ,
        0.90837502, 0.89887503],
       [0.96237499, 0.95714808, 0.94847095, 0.96102899, 0.94751745,
        0.93059283, 0.94691813, 0.93308383, 0.93211913, 0.89138132,
        0.911125  , 0.90966669],
       [0.94616666, 0.95696342, 0.9458223 , 0.96563643, 0.9549216 ,
        0.94216073, 0.96091771, 0.93814659, 0.91815042, 0.89042807,
        0.90616667, 0.90533337],
       [0.95631251, 0.96195048, 0.94437754, 0.96985984, 0.96559232,
        0.96157819, 0.96461207, 0.93836671, 0.93142569, 0.89495766,
        0.91945836,